In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max,lab_ph_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
201220,125078,66690,0,2106-04-27 01:47:50,2106-05-01 11:25:46,1,62.67646,1,NA,⋯,1,7.45,7.42,7.45,1,1,0,1,0.4496715,1.817096
215842,151232,11663,0,2188-02-14 01:48:15,2188-02-15 19:02:48,1,86.76186,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.1577062,1.187234
234312,164444,86645,1,2165-06-22 01:47:16,2165-07-07 13:55:20,1,56.08904,1,NA,⋯,1,7.49,7.49,7.49,1,0,0,0,0.3034738,1.435696
289157,146726,10304,0,2156-06-23 14:26:00,2156-06-30 09:26:00,1,45.91093,1,2156-06-25,⋯,1,7.44,7.44,7.44,1,0,1,0,0.7020434,1.424413
211964,160170,94534,0,2160-03-05 14:23:19,2160-03-06 22:48:41,1,59.38693,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.3399883,1.515125
230173,112553,31544,0,2140-01-30 20:39:25,2140-02-02 18:41:39,1,91.50000,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2497070,1.332813


In [4]:
table_one <- data.table::fread(file.path(data_dir, "tableone_raw.csv"), data.table = FALSE)
head(table_one)

p_val,wtd_p_val,main_0,main_1,sub_0,sub_1,wtd_main_0,wtd_main_1,wtd_sub_0,wtd_sub_1,var
4.472893e-02,7.861962e-02,66.68924,65.824051,17.207039,16.620088,65.444828,64.654880,17.014068,16.466339,age
1.416420e-02,1.507728e-02,50.82285,47.665687,NA,NA,47.089492,43.841849,NA,NA,female
6.415559e-11,1.909584e-13,78.56295,82.979049,23.579777,26.702495,85.637556,91.568834,27.685863,32.234984,weight
1.705518e-15,4.041212e-14,8.50597,10.045054,7.451124,7.681609,8.332934,9.888314,7.564772,7.744827,elix_score
1.378591e-15,8.274714e-12,19.63246,20.778975,5.788347,5.446293,19.793740,20.807581,5.667359,5.364432,saps
7.477319e-27,0.000000e+00,5.30526,6.323865,3.618029,3.796365,5.528346,6.510264,3.764854,3.812443,sofa


In [5]:
wpv <- table_one %>%
    filter(wtd_p_val < 0.05) %>%
    pull(var) %>%
    discard(grepl("day", .)) %>%
    gsub("female", "gender", .)
wpv

[1] "gender"                     "weight"                    
 [3] "elix_score"                 "saps"                      
 [5] "sofa"                       "vent"                      
 [7] "vaso"                       "sedative"                  
 [9] "icd_chf"                    "icd_afib"                  
[11] "icd_renal"                  "icd_copd"                  
[13] "icd_cad"                    "icd_stroke"                
[15] "icd_malignancy"             "vs_heart_rate_first"       
[17] "vs_temp_first"              "lab_platelet_first"        
[19] "lab_sodium_first"           "lab_potassium_first"       
[21] "lab_chloride_first"         "lab_bun_first"             
[23] "lab_creatinine_first"       "lab_po2_first"             
[25] "lab_pco2_first"             "lab_bnp_flag"              
[27] "lab_troponin_flag"          "lab_creatinine_kinase_flag"

In [6]:
design_echo_ps_ate <- svydesign(ids = ~ icustay_id, weights = ~ ps_weight, data = full_data)

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + gender + weight + elix_score + saps + sofa + vent + vaso + sedative + icd_chf + icd_afib + icd_renal + icd_copd + icd_cad + icd_stroke + icd_malignancy + vs_heart_rate_first + vs_temp_first + lab_platelet_first + lab_sodium_first + lab_potassium_first + lab_chloride_first + lab_bun_first + lab_creatinine_first + lab_po2_first + lab_pco2_first + lab_bnp_flag + lab_troponin_flag + lab_creatinine_kinase_flag"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = design_echo_ps_ate)

Survey design:
svydesign(ids = ~icustay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -1.1361178  1.7634941  -0.644 0.519459    
echo1                       -0.3227281  0.0919478  -3.510 0.000454 ***
gender1                      0.1286428  0.0935584   1.375 0.169217    
weight                      -0.0113949  0.0020834  -5.469 4.82e-08 ***
elix_score                   0.0295388  0.0067338   4.387 1.18e-05 ***
saps                         0.0777065  0.0114032   6.814 1.10e-11 ***
sofa                         0.1692136  0.0172440   9.813  < 2e-16 ***
vent1                        0.1025827  0.1533943   0.669 0.503697    
vaso1                        0.0516015  0.1066234   0.484 0.628444    
sedative1                   -0.3916552  0.1297696  -3.018 0.002562 ** 
icd_chf1                 

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.3210630,0.01012748,10.1783920
echo1,0.7241708,0.60474856,0.8671757
gender1,1.1372838,0.94674237,1.3661736
weight,0.9886697,0.98464077,0.9927152
elix_score,1.0299794,1.01647503,1.0436631
saps,1.0808054,1.05691742,1.1052333
sofa,1.1843731,1.14501287,1.2250863
vent1,1.1080289,0.82031685,1.4966510
vaso1,1.0529560,0.85438221,1.2976819
sedative1,0.6759372,0.52413870,0.8716988
